# CKAN Database

The CKAN Database is a commonly used and world renowned database where governments all around the world store their data. This notebook will serve as a means to parse through the api. 
</b>


**Information on the api can be found here:** [ckan api guide](https://docs.ckan.org/en/latest/api/#example-importing-datasets-with-the-ckan-api). General information on the ckan database and its participants can be found here: [ckan official website](https://ckan.org).

requests documentation: [here](https://requests.readthedocs.io/en/latest/)

pandas documentation: [here](https://pandas.pydata.org/docs/)

In [1]:
import requests
import pandas as pd

## Using Requests to access the API

The code below gives an example of using requests to pull from an api, as well as give an example of generally how this data is unpacked.

In [2]:
ckan_url = "http://catalog.data.gov/api/3/action/package_list"

api_token = "45tYhqFq71zd3xYo29eMgLESXiNml4Xxm9JfMmTl"

auth = {
    'X-Api-Key': api_token
}

response = requests.get(url = ckan_url,
                        headers = auth)

assert response.status_code == 200

In [3]:
print(response.status_code)

200


The particular request above gets the data catalog for all data.gov publicly available datasets, the catalog itself looks like this:

In [4]:
response_dict = response.json()

This is a lot of info, we probably want to see the keys, and maybe even just a list of the dataset, sourcename, and url, we can do this by first looking at the column names

In [5]:
response_dict.keys()

dict_keys(['help', 'success', 'result'])

### Note: To get help...

In [8]:
help_url = response_dict['help']

help_response = requests.get(url = help_url,
                           headers = auth)

assert help_response.status_code == 200

print(help_response.json()['result'].replace('`','\''))


    Searches for packages satisfying a given search criteria.

    This action accepts solr search query parameters (details below), and
    returns a dictionary of results, including dictized datasets that match
    the search criteria, a search count and also facet information.

    **Solr Parameters:**

    For more in depth treatment of each paramter, please read the
    'Solr Documentation
    <https://lucene.apache.org/solr/guide/6_6/common-query-parameters.html>'_.

    This action accepts a *subset* of solr's search query parameters:


    :param q: the solr query.  Optional.  Default: ''"*:*"''
    :type q: string
    :param fq: any filter queries to apply.  Note: ''+site_id:{ckan_site_id}''
        is added to this string prior to the query being executed.
    :type fq: string
    :param fq_list: additional filter queries to apply.
    :type fq_list: list of strings
    :param sort: sorting of the search results.  Optional.  Default:
        '''score desc, metadata_modified 

### Continuing with unpacking datasets using the Solr query parameters

In [8]:
class TooMuchError(Exception):
    pass
# This function will be used to render api data in chunks
def getLimits(url):
    response = requests.get(url,
                           headers = auth)
    print(response.headers)
    rate_limits = {
        'rate_limit': response.headers.get("X-RateLimit-Limit"),
        'rate_limit_remaining': response.headers.get("X-RateLimit-Remaining"),
        'rate_limit_reset':response.headers.get("X-RateLimit-Reset")
    }
    return rate_limits

def ckanifyTokens(params):
    ckanifyed = "&".join([key+"="+str(params[key]) for key in params.keys()])
    return ckanifyed

def hitMetadata(url=ckan_url, 
                auth=auth,
                results=[],
                **params):
    #init params
    base_params = {
        'rows': 15,
        'wt':'python'
    }
    params = {**params,**base_params}
    #ckanify the tokens 
    params_encoded = ckanifyTokens(params)
    #init start
    if 'start' not in params.keys():
        params['start'] = 0
    #get request
    response = requests.get(url = url, 
                            params = params_encoded,
                            headers = auth)
    #pass status
    assert response.status_code == 200
    #get json dump
    json_dump = response.json()
    # raise error if query is too large
    if json_dump['result']['count']>1000:
        msg = ''' your query is gathering '''+str(json_dump['result']['count'])+ ''' datasets, 
        consider rewriting your query to look for a topic more specific than your current query: 
        \"'''+params['q']+'''\"'''
        raise TooMuchError(msg)
        
    #paginate
    params['start'] += params['rows']
    #concat results 
    results += json_dump['result']['results']
    # get the number of records left
    records_left =  json_dump['result']['count'] - params['start']
    #continue recursion if necessary
    if records_left > 0:
        print("Intiating recusion to fetch "+ str(records_left)+' remaining records')   
        hitMetadata(results=results,**params)
    return results
    

### Unpacking the unique resources

Using the metadata api, we are now going to extract all of the datasets, we are going to use something small because as you will see, the sheer number of datasets gets pretty large pretty fast...

In [9]:
# look for a subject to find a dataset
query = "aliens"

#find metadata in data.gov for the subsequent datasets
metajson = hitMetadata(q=query)

print(metajson[:3])

Intiating recusion to fetch 47 remaining records
Intiating recusion to fetch 32 remaining records
Intiating recusion to fetch 17 remaining records
Intiating recusion to fetch 2 remaining records
[{'author': None, 'author_email': None, 'creator_user_id': '1ecd1fb1-1be6-46bb-b90d-07a0762ed104', 'id': 'bbca377b-c941-477c-98b1-4935cb9889dd', 'isopen': False, 'license_id': 'us-pd', 'license_title': 'us-pd', 'maintainer': 'Bureau of Indian Affairs Branch of Geospatial Support', 'maintainer_email': 'geospatial@bia.gov', 'metadata_created': '2023-09-25T14:34:25.539723', 'metadata_modified': '2023-11-28T21:12:17.313523', 'name': 'bia-tracts', 'notes': 'The BIA Tract Viewer allows the user to easily access an interactive map displaying the details for tracts, parcels, and other relevant BIA lands. This viewer provides tools to assist with research, planning, and reporting. This document provides guidelines for using the app. On July 5, 2023 Office of Trust Services/ Branch of Geospatial Support 

Now we will translate the results from json to a pandas dataframe so it is easier to read.

In [10]:
def resultsToPandas(json):
    common_keys = set.intersection(*map(set, json))
    transformed = {k: [dic[k] for dic in json] for k in common_keys}
    return pd.DataFrame(transformed)

metadata_df = resultsToPandas(metajson)

for n in metadata_df['name']:
    print(n)

bia-tracts
department-of-defense-dod
records-about-japanese-americans-relocated-during-world-war-ii
illegal-immigration-and-crime-in-san-diego-and-el-paso-counties-1985-1986-9fc89
workload-and-adjudication-statistics
invasive-plant-inventory-at-farallon-island-national-wildlife-refuge-data-documentation
invasive-plant-prioritization-for-inventory-and-early-detection-at-desert-and-pahranagat-n
invasive-plant-prioritization-for-inventory-and-early-detection-at-farallon-island-nationa
invasive-plant-prioritization-for-inventory-and-early-detection-at-ruby-lake-national-wild
invasive-plant-prioritization-for-inventory-and-early-detection-at-stillwater-national-wil
data-collected-to-determine-the-relationship-between-coral-reef-health-alien-algae-and-fish-bio
invasive-plant-prioritization-for-inventory-and-early-detection-at-lower-klamath-and-tule-
data-from-release-and-establishment-of-the-weevil-mecinus-janthiniformis-for-biological-co
invasive-plant-inventory-at-kern-national-wildlife-re

It seems that the data is stored under the "resources" tag, so lets now unpack that

In [14]:
resources = metadata_df['resources']

all_resources = []

for r in resources:
    all_resources+=r

all_resources_df = resultsToPandas(all_resources)

all_resources_df.drop_duplicates(subset=['url'])

,mimetype,id,size,description,hash,position,url_type,package_id,name,cache_last_updated,format,mimetype_inner,state,cache_url,last_modified,metadata_modified,resource_type,url,created
0,text/html,85d0e0a3-0110-4ce8-8f25-9ed8ed02a500,None,,,0,None,bbca377b-c941-477c-98b1-4935cb9889dd,ArcGIS Hub Dataset,None,HTML,None,active,None,None,2023-09-25T14:34:25.536081,None,https://opendata-1-bia-geospatial.hub.arcgis.c...,2023-09-25T14:34:25.543527
1,application/json,48300a1f-2dbd-49ab-b3f6-61701f2aa995,None,,,1,None,bbca377b-c941-477c-98b1-4935cb9889dd,ArcGIS GeoService,None,ArcGIS GeoServices REST API,None,active,None,None,2023-09-25T14:34:25.535911,None,https://biamaps.geoplatform.gov/biatracts/,2023-09-25T14:34:25.543523
2,,e3ccb294-022c-42af-b0db-63296d2b0e5b,None,,,0,None,3f656057-df3d-4d3f-a810-781ec644d930,Catalog description for the Japanese-American ...,None,,None,active,None,None,2020-11-12T14:59:18.905463,None,https://catalog.archives.gov/id/1264228,2020-11-12T14:59:18.905463
3,application/octet-stream,0c2a3d21-71e9-42a8-ae13-4bce36d1d524,None,,,1,None,3f656057-df3d-4d3f-a810-781ec644d930,Japanese-American Internee Data File,None,BIN,None,active,None,None,2020-11-12T14:59:18.905474,None,https://catalog.archives.gov/OpaAPI/media/1264...,2020-11-12T14:59:18.905474
4,,e2b4b28e-f5f9-4b08-b96c-f7c8e22c7ede,None,ICPSR09330.v1,,0,None,fbf23125-eb0b-4475-a5eb-1e25e0f23937,Illegal Immigration and Crime in San Diego and...,None,,None,active,None,None,2023-02-13T19:23:52.724591,None,https://doi.org/10.3886/ICPSR09330.v1,2021-08-18T21:07:15.015139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,application/http,ac1fece8-a665-49fe-b6e2-c96b97caa7c5,None,Landing page for access to the data,,0,None,6b9f915e-5141-480a-b498-9d65fd807e8a,Digital Data,None,XML,None,active,None,None,2023-06-01T12:56:56.987896,None,https://doi.org/10.5066/F7WQ02JX,2023-06-01T12:56:57.023657
259,text/xml,b90e4490-2f4f-4748-856d-f8e5f0b41540,None,The metadata original format,,1,None,6b9f915e-5141-480a-b498-9d65fd807e8a,Original Metadata,None,XML,None,active,None,None,2023-10-27T21:59:48.255425,None,https://data.usgs.gov/datacatalog/metadata/USG...,2023-06-01T12:56:57.023653
260,application/zip,43ef72bb-ac20-46cd-a1cb-67b6bc5e6fac,None,A geodatabase containing spatially-reference i...,,0,None,711d1485-18a3-4537-84fb-c33735075c8a,ELS_InvasivePlantInventoryData_2019.zip,None,ZIP,None,active,None,None,2023-06-01T15:16:07.661913,None,https://ecos.fws.gov/ServCat/DownloadFile/2041...,2023-06-01T15:16:07.715009
261,application/json,4390804d-91b2-4b1f-a5af-9c60e8477f00,None,,,0,None,12c36cb3-6934-4ca3-91b9-6bd3feec3e3b,ArcGIS GeoService,None,ArcGIS GeoServices REST API,None,active,None,None,2023-08-30T17:00:14.578089,None,https://bia-geospatial.maps.arcgis.com/apps/we...,2023-08-30T17:00:14.600182


I have added a method to follow these links and eventually get the data, for the purposes of this book we will not be doing this here, but source code will be developed to use this data in the near future, this is just meant to be a browser...

In [42]:
print([link for link in list(all_resources_df['url'])])

['https://opendata-1-bia-geospatial.hub.arcgis.com/documents/BIA-Geospatial::bia-tracts', 'https://biamaps.geoplatform.gov/biatracts/', 'https://catalog.archives.gov/id/1264228', 'https://catalog.archives.gov/OpaAPI/media/1264228/content/electronic-records/rg-210/wra/WRA.FORM26.PU?download=true', 'https://doi.org/10.3886/ICPSR09330.v1', 'https://www.justice.gov/eoir/page/file/1139516/download', 'https://www.justice.gov/eoir/page/file/1061521/download', 'https://www.justice.gov/eoir/page/file/1296536/download', 'https://www.justice.gov/eoir/page/file/1163611/download', 'https://www.justice.gov/eoir/page/file/1248506/download', 'https://www.justice.gov/eoir/page/file/1107716/download', 'https://www.justice.gov/eoir/page/file/1248491/download', 'https://www.justice.gov/eoir/page/file/1107366/download', 'https://www.justice.gov/eoir/page/file/1062976/download', 'https://www.justice.gov/eoir/page/file/1111476/download', 'https://www.justice.gov/eoir/page/file/1248501/download', 'https://www